In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# *SPARK STREAMING*

# 1 aplikacja

Spark Streaming potrzebuje minium 2 rdzenie.

StreamingContext (tworzony na podstawie SparkContextu) reprezentuje połączenie z klastrem i służy do tworzenia DStreamów

In [43]:
sc = SparkContext("local[2]", "NetworkWC")
ssc = StreamingContext(sc,1)

In [44]:
sc.uiWebUrl

'http://192.168.165.131:4040'

In [45]:
sc.parallelize(range(10)).map(lambda x: x*2).collect()

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [46]:
lines = ssc.socketTextStream("localhost", 9999)
#words = lines.flatMap(lambda line: line.split(" "))
words = lines.flatMap(lambda line: line.split(" ")).filter(lambda x: x != "" and not x.isdigit())

In [47]:
wordcounts = words.map(lambda word: (word.lower().strip("?!,.-;:"),1)).reduceByKey(lambda x,y: x+y)

In [48]:
#wordcounts = words.map(lambda word: (word,1)).reduceByKey(lambda x,y: x+y)


In [49]:
wordcounts.pprint()

Obliczenia zaczynają się dopiero od wywołania `start`

In [51]:
ssc.start()
ssc.awaitTermination(15)
ssc.stop(True,True)

Py4JJavaError: An error occurred while calling o3133.start.
: java.lang.IllegalStateException: StreamingContext has already been stopped
	at org.apache.spark.streaming.StreamingContext.start(StreamingContext.scala:608)
	at org.apache.spark.streaming.api.java.JavaStreamingContext.start(JavaStreamingContext.scala:556)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:280)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run

> **TODO**: Popraw powyższą aplikację tak aby printowane były oczyszczone słowa

# 2 aplikacja

Aby Spark Streaming mógł łączyć dane z wielu batchy (stateful transformations) konieczne jest wskazanie lokalizacji gdzie zapisywane będą checkpointy.

In [52]:
sc = SparkContext("local[2]", "NetworkWC")
ssc = StreamingContext(sc, 1)
ssc.checkpoint("tmp")

In [53]:
lines = ssc.socketTextStream("localhost", 9999)

In [54]:
def updateFunc(newValues, runningCount):
    if runningCount is None:
        runningCount = 0
    return sum(newValues, runningCount)

In [55]:
words = lines.flatMap(lambda line: line.split(" "))
pairs = words.map(lambda word: (word, 1))

In [56]:
runningCounts = pairs.updateStateByKey(updateFunc)
runningCounts.pprint()

In [58]:
ssc.start()
ssc.awaitTermination(60)
ssc.stop(True,True)

Py4JJavaError: An error occurred while calling o4019.start.
: java.lang.IllegalStateException: StreamingContext has already been stopped
	at org.apache.spark.streaming.StreamingContext.start(StreamingContext.scala:608)
	at org.apache.spark.streaming.api.java.JavaStreamingContext.start(JavaStreamingContext.scala:556)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:280)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run

> **TODO**: Popraw powyższą aplikację tak aby printowane były oczyszczone słowa, spraw aby `początkowy stan` zawierał wpisy: ('moby', 10) oraz ('dick', 10) (opcja w updateStateByKey)

In [59]:
sc = SparkContext("local[2]", "NetworkWC")
ssc = StreamingContext(sc, 1)
ssc.checkpoint("tmp")

In [60]:
lines = ssc.socketTextStream("localhost", 9999)
start = sc.parallelize([("moby",10),("dick",10)])

In [61]:
def updateFunc(newValues, runningCount):
    if runningCount is None:
        runningCount = 0
    return sum(newValues, runningCount)

In [63]:
words = lines.flatMap(lambda line: line.split(" ")).filter(lambda x: x != "" and not x.isdigit())
pairs = words.map(lambda word: (word.lower().strip("?!.,;:-"), 1))

In [66]:
runningCounts = pairs.updateStateByKey(updateFunc, initialRDD=start)
runningCounts.pprint()

In [68]:
ssc.start()
ssc.awaitTermination(30)
ssc.stop(True, True)

Py4JJavaError: An error occurred while calling o9025.start.
: java.lang.IllegalStateException: StreamingContext has already been stopped
	at org.apache.spark.streaming.StreamingContext.start(StreamingContext.scala:608)
	at org.apache.spark.streaming.api.java.JavaStreamingContext.start(JavaStreamingContext.scala:556)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:280)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run

# 3 aplikacja

In [82]:
sc = SparkContext("local[2]", "NetworkWC")
ssc = StreamingContext(sc, 1)
ssc.checkpoint("tmp")

In [83]:
lines = ssc.socketTextStream("localhost", 9999)

In [84]:
words = lines.flatMap(lambda line: line.split(" ")).flatMap(lambda line: line.split("—"))\
.filter(lambda x: x != "" and not x.isdigit())

In [85]:
pairs = words.map(lambda word: (word.lower().strip("?!,.-;:"),1))
windowedWordCounts = pairs.reduceByKeyAndWindow(lambda x, y: x + y, lambda x, y: x - y, 30, 10)
windowedWordCounts.pprint()

In [86]:
ssc.start()
ssc.awaitTermination(60)
ssc.stop(True,True)

-------------------------------------------
Time: 2019-01-26 13:57:18
-------------------------------------------
('moby', 1)
('chapter', 1)
('1', 1)
('call', 1)
('years', 1)
('ago', 1)
('never', 1)
('mind', 1)
('long', 1)
('precisely', 1)
...

-------------------------------------------
Time: 2019-01-26 13:57:28
-------------------------------------------
('moby', 1)
('chapter', 1)
('1', 1)
('call', 1)
('years', 1)
('ago', 1)
('never', 1)
('mind', 1)
('long', 1)
('precisely', 1)
...

-------------------------------------------
Time: 2019-01-26 13:57:38
-------------------------------------------
('moby', 1)
('chapter', 1)
('1', 1)
('call', 1)
('years', 1)
('ago', 1)
('never', 1)
('mind', 1)
('long', 1)
('precisely', 1)
...

-------------------------------------------
Time: 2019-01-26 13:57:48
-------------------------------------------
('i', 9)
('thought', 1)
('would', 1)
('watery', 1)
('of', 12)
('world', 1)
('is', 7)
('way', 1)
('have', 1)
('regulating', 1)
...

--------------------

> **TODO**: Zmodyfikuj powyższą aplikację tak aby printowane były county **słów** dłuższych oraz krótszych od 4

In [87]:
sc = SparkContext("local[2]", "NetworkWC")
ssc = StreamingContext(sc, 1)
ssc.checkpoint("tmp")

In [88]:
lines = ssc.socketTextStream("localhost", 9999)

In [89]:
words = lines.flatMap(lambda line: line.split(" ")).flatMap(lambda line: line.split("—"))\
.filter(lambda x: x != "" and not x.isdigit())

In [90]:
pairs = words.map(lambda word: word.lower().strip("?!.,;:-"))\
.map(lambda word: (">4" if len(word) > 4 else "<=4", 1))
windowedWordCounts = pairs.reduceByKeyAndWindow(lambda x, y: x + y, lambda x, y: x - y, 30, 10)
windowedWordCounts.pprint()

In [91]:
ssc.start()
ssc.awaitTermination(60)
ssc.stop(True,True)

-------------------------------------------
Time: 2019-01-26 14:02:06
-------------------------------------------
('<=4', 20)
('>4', 14)

-------------------------------------------
Time: 2019-01-26 14:02:16
-------------------------------------------
('<=4', 101)
('>4', 61)

-------------------------------------------
Time: 2019-01-26 14:02:26
-------------------------------------------
('<=4', 164)
('>4', 107)

-------------------------------------------
Time: 2019-01-26 14:02:36
-------------------------------------------
('<=4', 205)
('>4', 134)

-------------------------------------------
Time: 2019-01-26 14:02:46
-------------------------------------------
('<=4', 189)
('>4', 127)

-------------------------------------------
Time: 2019-01-26 14:02:56
-------------------------------------------
('<=4', 207)
('>4', 115)



# 4 aplikacja

In [3]:
sc = SparkContext("local[2]", "NetworkWC")
ssc = StreamingContext(sc, 1)

`words.txt` to przetowrzony [Opinion Lexicon](https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html#lexicon)

In [93]:
sentyment = sc.textFile("words.txt").map(lambda x: x.split(",")).map(lambda x: (x[0],x[1]))

In [94]:
lines = ssc.socketTextStream("localhost", 9999)
words = lines.flatMap(lambda line: line.split(" ")).flatMap(lambda line: line.split("—"))\
.filter(lambda x: x != "" and not x.isdigit())

In [95]:
wordcounts = words.map(lambda word: (word.lower().strip("?!,.-;:"),1)).reduceByKey(lambda x,y: x+y)

In [96]:
WCS = wordcounts.transform(lambda rdd: sentyment.rightOuterJoin(rdd))
WCS.pprint()

In [109]:
ssc.start()
ssc.awaitTermination(120)
ssc.stop(True,True)

Py4JJavaError: An error occurred while calling o28013.start.
: java.lang.IllegalStateException: StreamingContext has already been stopped
	at org.apache.spark.streaming.StreamingContext.start(StreamingContext.scala:608)
	at org.apache.spark.streaming.api.java.JavaStreamingContext.start(JavaStreamingContext.scala:556)
	at sun.reflect.GeneratedMethodAccessor63.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:280)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:748)


> **TODO**: Zmodyfikuj powyższą aplikację tak aby zwracała liczbę wystąpień słów pozytywnych, negatywnych i neutralnych

In [110]:
sc = SparkContext("local[2]", "NetworkWC")
ssc = StreamingContext(sc, 1)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=NetworkWC, master=local[2]) created by __init__ at <ipython-input-98-f15fae5a1f94>:1 

In [4]:
sentyment = sc.textFile("words.txt").map(lambda x: x.split(",")).map(lambda x: (x[0],x[1]))

In [5]:
lines = ssc.socketTextStream("localhost", 9999)
words = lines.flatMap(lambda line: line.split(" ")).flatMap(lambda line: line.split("—"))\
.filter(lambda x: x != "" and not x.isdigit())

In [6]:
pairs = words.map(lambda word: (word.lower().strip("?!,.-;:"),1))\
.transform(lambda rdd: sentyment.rightOuterJoin(rdd))
PS = pairs.map(lambda x: (x[1][0] if x[1][0] != None else "neutral", x[1][1])).reduceByKey(lambda x,y: x+y)

In [8]:
PS.pprint()

In [ ]:
ssc.start()
ssc.awaitTermination(120)
ssc.stop(True,True)

# *SPARK STUCTURED STREAMING*

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, lower, udf, window, min, max, avg

# 1 aplikacja

In [2]:
spark = SparkSession.builder.appName("Structured").getOrCreate()

In [3]:
lines = spark.readStream.format("socket").option("host", "localhost").option("port", 9999).load()

In [7]:
words = lines.select(explode(split(lines.value, " ")).alias("word"))
wordCounts = words.groupBy("word").count()

In [5]:
query = wordCounts.writeStream.outputMode("complete").format("console").start()
query.awaitTermination(60)
query.stop()

> **TODO**: Zmodyfikuj powyższą aplikację tak aby zwracała liczbę wystąpień oczyszczonych **słów** (konieczne będzie zastosowanie udf)

In [8]:
spark = SparkSession.builder.appName("Structured").getOrCreate()

In [9]:
lines = spark.readStream.format("socket").option("host", "localhost").option("port", 9999).load()

In [10]:
def cleaning(string):
    x = string.strip("?!.,;:-")
    if x.isdigit():
        out = ""
    else:
        out = x
    return out

cleaningUDF = udf(cleaning)

In [11]:
one = lines.select(explode(split(lower(lines.value), " ")).alias("word"))
two = one.select(cleaningUDF(one.word).alias("word"))
words = two.filter(two.word != "")

wordCounts = words.groupby("word").count()

In [12]:
words = lines.select(explode(split(lines.value, " ")).alias("word"))
wordCounts = words.groupBy("word").count()

In [13]:
query = wordCounts.writeStream.outputMode("complete").format("console").start()
query.awaitTermination(60)
query.stop()

# 2 aplikacja

Tak wyglądać powinien schemat danych:

In [ ]:
#from pyspark.sql.types import *
#schemat = StructType([
#    StructField("age", DoubleType(), True),
#    StructField("workclass", StringType(), True),
#    StructField("fnlwgt", DoubleType(), True),
#    StructField("education", StringType(), True),
#    StructField("education-num", DoubleType(), True),
#    StructField("marital-status", StringType(), True),
#    StructField("occupation", StringType(), True),
#    StructField("relationship", StringType(), True),
#    StructField("race", StringType(), True),
#    StructField("sex", StringType(), True),
#    StructField("capital-gain", DoubleType(), True),
#    StructField("capital-loss", DoubleType(), True),
#    StructField("hours-per-week", DoubleType(), True),
#    StructField("native-country", StringType(), True),
#    StructField("target", StringType(), True)#
#])

In [14]:
spark = SparkSession.builder.appName("Structured").getOrCreate()

In [15]:
lines = spark.readStream.format("socket").option("host", "localhost").option("port", 9998).load()
lin = lines.withColumn("value", split(lines.value, ", "))
linn = lin.select(
    lin.value.getItem(0).cast("float").alias("age"),
    lin.value.getItem(1).alias("workclass"),
    lin.value.getItem(2).cast("float").alias("fnlwgt"),
    lin.value.getItem(3).alias("education"),
    lin.value.getItem(4).cast("float").alias("education-num"),
    lin.value.getItem(5).alias("marital-status"),
    lin.value.getItem(6).alias("occupation"),
    lin.value.getItem(7).alias("relationship"),
    lin.value.getItem(8).alias("race"),
    lin.value.getItem(9).alias("sex"),
    lin.value.getItem(10).cast("float").alias("capital-gain"),
    lin.value.getItem(11).cast("float").alias("capital-loss"),
    lin.value.getItem(12).cast("float").alias("hours-per-week"),
    lin.value.getItem(13).alias("native-country"),
    lin.value.getItem(14).alias("target"))
lineCounts = linn.groupBy('workclass').avg("age", "hours-per-week")

In [16]:
query = lineCounts.writeStream.outputMode("complete").format("console").start()
query.awaitTermination(60)
query.stop()

> **TODO**: Napisz aplikację liczącą średnią "capital-gain" i "capital-loss" dla płci i edukacji

# 3 aplikacja

In [ ]:
spark = SparkSession.builder.appName("Structured").getOrCreate()

In [ ]:
lines = spark.readStream.format("socket").option("host", "localhost").option("port", 9998)\
.option("includeTimestamp", True).load()
lin = lines.withColumn("value", split(lines.value, ", "))
linn = lin.select(
    lin.value.getItem(0).cast("float").alias("age"),
    lin.value.getItem(1).alias("workclass"),
    lin.value.getItem(2).cast("float").alias("fnlwgt"),
    lin.value.getItem(3).alias("education"),
    lin.value.getItem(4).cast("float").alias("education-num"),
    lin.value.getItem(5).alias("marital-status"),
    lin.value.getItem(6).alias("occupation"),
    lin.value.getItem(7).alias("relationship"),
    lin.value.getItem(8).alias("race"),
    lin.value.getItem(9).alias("sex"),
    lin.value.getItem(10).cast("float").alias("capital-gain"),
    lin.value.getItem(11).cast("float").alias("capital-loss"),
    lin.value.getItem(12).cast("float").alias("hours-per-week"),
    lin.value.getItem(13).alias("native-country"),
    lin.value.getItem(14).alias("target"),
    lin.timestamp.alias("timestamp"))
lineCounts = linn.groupBy(window("timestamp", "20 seconds", "10 seconds")).avg("hours-per-week")

In [ ]:
query = lineCounts.writeStream.outputMode("complete").format("console").start()
query.awaitTermination(60)
query.stop()

> **TODO**: Napisz aplikację liczącą średnią, maximum i minimum "hours-per-week" dla rasy w oknach 20 sekundowych z krokiem 10 sekundowym